## Joining of datasets to obtain full weekly data
Joining population and location data to the protest dataset. The output format has all the locations for each week, which is then populated if a protest took place. Data is for 01-Jan-2020 to 20-May-2022 (week 20)

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
coords = pd.read_csv('data/USDATA_HBCU.csv')
urban_pop = pd.read_csv('data/US_Census_Urban_Area_ACSDP5Y2020.DP05-2022-06-14T153824.csv')
protests = pd.read_csv('data/acled_locationMatched.csv')
protests = protests[(protests['EVENT_TYPE'] == 'Protests') | (protests['EVENT_TYPE'] == 'Riots')]

In [12]:
coords = coords.drop(['Unnamed: 0'], axis=1)

Urban area data contains three rows per location, here we are keeping only the one which contains the data we need

In [13]:
urban_pop['Label'] = urban_pop['Label'].shift(2)
urban_pop['Sex ratio (males per 100 females)'] = urban_pop['Sex ratio (males per 100 females)'].shift(1)
urban_pop['Median age (years)'] = urban_pop['Median age (years)'].shift(1)
urban_pop['Total housing units'] = urban_pop['Total housing units'].shift(1)
 
to_keep = [(i*3)+2 for i in range(0,3592)]
indices = list(range(0, 10776))
to_drop = list(set(indices) - set(to_keep))

urban_pop = urban_pop.drop(to_drop)

In [14]:
urban_pop['Label'] = urban_pop['Label'].str[:-6]
urban_pop['Location'] = urban_pop['Label']
urban_pop['Location'] = urban_pop['Location'].str.replace('-', '')
urban_pop['Location'] = urban_pop['Location'].str.replace(',', '')
urban_pop['Location'] = urban_pop['Location'].str.replace(' ', '')

In [15]:
coords['Location'] = coords['NAMELSAD10']
coords['Location'] = coords['Location'].str.replace('--', '')
coords['Location'] = coords['Location'].str.replace('-', '')
coords['Location'] = coords['Location'].str.replace(',', '')
coords['Location'] = coords['Location'].str.replace(' ', '')

In [16]:
merged_data = pd.merge(coords,urban_pop,on='Location', how='left')
merged_data.to_csv('data/population_data.csv')

# Protest Data

In [17]:
protests['Date'] = pd.to_datetime(protests['EVENT_DATE'])

In [18]:
protests['Year-Week'] = protests['Date'].dt.strftime('%Y-%U')

In [19]:
weekly_df = pd.DataFrame(columns=merged_data.columns.to_list())

In [20]:
# Only needs to go up to week 20 for 2022
for year in ['2020', '2021']:
    for i in range(53):
        df = merged_data.copy()
        df['Year-Week'] = year + '-' + str(i)
        weekly_df = weekly_df.append(df)
        
for i in range(21):
        df = merged_data.copy()
        df['Year-Week'] = '2022' + '-' + str(i)
        weekly_df = weekly_df.append(df)

In [21]:
protests = protests.drop(['Unnamed: 0'], axis=1)
protests = protests.drop(['X'], axis=1)
protests['Protest?'] = 1

## All protests

In [22]:
weekly_data = pd.merge(weekly_df,protests,on=['UACE10', 'GEOID10', 'NAME10', 'NAMELSAD10', 'INTPTLAT10', 'INTPTLON10', 'Year-Week'], how='left')

In [23]:
weekly_data['Protest?'].fillna(0, inplace=True)

In [24]:
weekly_data = weekly_data.drop_duplicates(subset=['GEOID10', 'Year-Week'], keep='first')

In [16]:
weekly_data.to_csv('data/weekly_protests.csv')

## BLM only

In [17]:
blm = protests[protests['BLM'] == 1]

In [18]:
blm_protests = pd.merge(weekly_df,blm,on=['UACE10', 'GEOID10', 'NAME10', 'NAMELSAD10', 'INTPTLAT10', 'INTPTLON10', 'Year-Week'], how='left')
blm_protests['Protest?'].fillna(0, inplace=True)
blm_protests = blm_protests.drop_duplicates(subset=['GEOID10', 'Year-Week'], keep='first')

blm_protests.to_csv('data/blm_weekly_protests.csv')

## Alt-Right only

In [19]:
alt_right = protests[protests['Alt.Right'] == 1]

In [20]:
alt_right_protests = pd.merge(weekly_df,alt_right,on=['UACE10', 'GEOID10', 'NAME10', 'NAMELSAD10', 'INTPTLAT10', 'INTPTLON10', 'Year-Week'], how='left')
alt_right_protests['Protest?'].fillna(0, inplace=True)
alt_right_protests = alt_right_protests.drop_duplicates(subset=['GEOID10', 'Year-Week'], keep='first')

alt_right_protests.to_csv('data/alt-right_weekly_protests.csv')

# Merged daily protest data

In [25]:
columns_format = ['20 to 24 years', '25 to 34 years', '65 years and over', 'Male', 'Black or African American', 
              'White', 'One race', 'Total population', 'Sex ratio (males per 100 females)', 'Median age (years)', 
                'Total housing units']
for column in columns_format:
    merged_data[column] = merged_data[column].replace('[%\,]', '', regex=True).astype(float)

In [26]:
protests_rel = protests[protests['Actor.Affiliation'].str.contains('Other Actors')==False]
daily_protests = pd.merge(protests_rel, merged_data, on=['UACE10', 'GEOID10', 'NAME10', 'NAMELSAD10', 'INTPTLAT10', 'INTPTLON10'], how='left')
daily_protests.to_csv('data/daily_protest_pop.csv')

In [27]:
# Keeping in locations where protest didn't happen
all_locs = pd.merge(protests_rel, merged_data, on=['UACE10', 'GEOID10', 'NAME10', 'NAMELSAD10', 'INTPTLAT10', 'INTPTLON10'], how='right')
all_locs.to_csv('data/daily_all_locs.csv')

## First occurrence

In [62]:
first_occ = daily_protests.sort_values(by='EVENT_DATE')
first_occ = first_occ.drop_duplicates(subset='GEOID10', keep='first')
first_occ.to_csv('data/first_occurrence.csv')

## Locations with/without protest occurrence

In [63]:
occurrence = pd.merge(merged_data, first_occ.iloc[:, :49], on=['UACE10', 'GEOID10', 'NAME10', 'NAMELSAD10', 'INTPTLAT10', 'INTPTLON10'], how='left')
occurrence[['Protest?']] = np.where(occurrence[['Protest?']].isnull(), 0, 1)
occurrence.to_csv('data/protest_occurrence.csv')

In [64]:
blm_occ = daily_protests[daily_protests['Actor.Affiliation'].str.contains('Alt-Right')==False]
blm_occ = pd.merge(merged_data, blm_occ.iloc[:, :49], on=['UACE10', 'GEOID10', 'NAME10', 'NAMELSAD10', 'INTPTLAT10', 'INTPTLON10'], how='left')
blm_first_occ = blm_occ.drop_duplicates(subset='GEOID10', keep='first')
blm_first_occ[['Protest?']] = np.where(blm_first_occ[['Protest?']].isnull(), 0, 1)
blm_first_occ.to_csv('data/blm_occurrence.csv')

In [65]:
right_occ = daily_protests[daily_protests['Actor.Affiliation'].str.contains('BLM')==False]
right_occ = pd.merge(merged_data, right_occ.iloc[:, :49], on=['UACE10', 'GEOID10', 'NAME10', 'NAMELSAD10', 'INTPTLAT10', 'INTPTLON10'], how='left')
right_occ = right_occ.drop_duplicates(subset='GEOID10', keep='first')
right_occ[['Protest?']] = np.where(right_occ[['Protest?']].isnull(), 0, 1)
right_occ.to_csv('data/right_occurrence.csv')